In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# Implementierung einer Künstlichen Intelligenz für das Spiel Othello

## Initiale Konfiguration

Importieren von Abhängigkeiten und Konfiguration

import math
import numpy as np
import random

In [ ]:
%run othello_game.ipynb

Die aktuellen Nutzen-Werte der beiden Spieler werden in einer globalen Variable gespeichert, sodass diese in der GUI angezeigt werden können. Die Werte werden in den entsprechenden Funktionen der Strategien aktualisiert.

In [ ]:
utilities = {WHITE: '-', BLACK: '-'}

## Heuristiken

Zum Abschätzen der Nützlichkeit eines Spielzustands wird eine Heuristik benötigt. Im folgenden sind einige solcher Heuristiken implementiert. Da Weiß der maximierende Spieler, und Schwarz der minimierende Spieler ist repräsentiert ein höherer Wert der Heuristik einen für Weiß vorteilhaften Zug, während ein niedriger Wert einen Vorteil für Schwarz repräsentiert. Die Werte der Heuristik liegen zwischen -1 und 1, wobei die Randwerte einen garantierten Sieg für den jeweiligen Spieler darstellen. Der Wert 0 steht für einen für beide Spieler gleich guten Spielzustand.

Da das Ziel des Spiels Othello ist, zum Ende des Spiels mehr Steine als der Gegner auf dem Spielfeld zu haben, ist es naheliegend, die Differez der Anzahlen von Steinen beider Spieler zur Abschätzung eines Zuges zu verwenden. Genau das macht die Disk-Count-Heuristik indem sie die Differenz der Steine, beider Spieler berechnet und den resultierenden Wert zur Normalisierung durch die maximale Anzahl an Steinen teilt. Bei genauerer Betrachtung ist es jedoch, gerade zu Beginn des Spiels nicht immer vorteilhaft, den Vorsprung an Steinen zu maximieren.

In [ ]:
def disc_count_heuristic(state):
    return (count_disks(state, WHITE) - count_disks(state, BLACK)) / 64

Eine weitere Heuristik ist die Mobilität der Spieler. Diese gibt an wie viele mögliche Züge ein Spieler gegenüber dessen Gegner machen kann. Die Idee bei dieser Heuristik ist, dass ein Spieler dadurch seine Freiheit maximiert, während die Freiheit des Gegners durch eine geringe Anzahl an Zügen eingeschränkt wird. Die Mobilitäts-Heuristik gibt an wie viele möglicher Züge mehr Weiß gegenüber Schwarz im Aktuellen Spielzustand hat. Auch dieser Wert wird durch Division durch die Anzahl an Feldern normalisiert, um die Grenzen von -1 und 1 einzuhalten.

In [ ]:
def mobility_heuristic(state):
    if state.turn == WHITE:
        return (len(state.possible_moves) -
                len(get_possible_moves(state, BLACK))) / 64
    else:
        return (len(get_possible_moves(state, WHITE)) -
                len(state.possible_moves)) / 64

Nicht nur die aktuelle, sondern auch die potentielle Mobilität kann vor allem in frühen Phasen des Spiels wichtig für die Bewertung einer Position sein. Die Funktion `pot_mob_heuristic` berechnet für einen Zustand `state` die Differenz der potentiellen Mobilität beider Spieler. Die potentielle Mobilität eines Spielers ist die Summe aller freien Felder um gegnerische Spielsteine, da Michael Buro dieses Merkmal als in seiner Dissertation als bestes ausgemacht hat. Das Ergebnis wird durch 3.5 geteilt, da es im Durchschnitt 3.5 Mal so viele potentielle Züge wie tatsächliche Züge gibt.

In [ ]:
def pot_mob_heuristic(state):
    board = list(state.board)
    fields = 0
    for (x,y) in state.frontier:
        for dx, dy in directions:
            xi = x + dx
            yi = y + dy
            if 0 <= xi < 8 and 0 <= yi < 8 and board[xi][yi] != 0:
                fields -= board[xi][yi]
    # Im Durchschnitt gibt es 3.5 Mal mehr potentielle wie tatsächliche Züge
    fields /= 3.5 
    return fields / 64

Die Funktion `combined_mobility_heuristic` kombiniert die aktuelle und potentielle Mobilität, wobei zu Beginn des Spiels die potentielle Mobilität stärker gewichtet wird und gegen Ende die aktuelle Mobilität. Michael Buro beschreibt in seiner Dissertation, dass die potentielle Mobilität bis 36 Spielsteine auf dem Feld liegen wichtiger für die Bewertung ist, als die aktuelle Mobilität. Dieser Sachverhalt soll durch eine lineare Kombination modelliert werden.

In [ ]:
def combined_mobility_heuristic(state):
    act = mobility_heuristic(state)
    pot = pot_mob_heuristic(state)
    return (1 - state.num_pieces / 50) * pot + (state.num_pieces / 50) *  act

Beim Spielen von Othello fällt auf, dass es bestimmte Felder gibt, deren Belegung von Vorteil ist, sowie einige, deren Belegung eher nachteilhaft ist. Diese Eigenschaft macht sich die Cowthello-Heuristik zu Nutze. Diese weist jedem Feld einen Wert zu der angibt, wie Vorteilhaft der Besitz dieses Feldes ist, bzw. wie Nachteilhaft die Belegung des Feldes durch den Gegner ist. Diese Gewichte werden dann mit der aktuellen Belegung des Spielfelds multipliziert und die Ergebnisse anschließend aufsummiert. Der resultierende Wert schätzt dann den Nutzen der aktuellen Position ein. Auch bei dieser Heuristik findet eine Normalisierung statt.

Aufgrund der Symmetrie des Othello Spielfeldes ist es für die Weight-Heuristik nicht nötig, für jedes Feld einzeln dessen Gewicht anzugeben. Stattdessen werden ausschließlich die Gewichte für ein Viertel des Spielfelds angegeben und dieses anschließend gespiegelt. Die Funktion `gen_cowthello_matrix` generiert dann die Gewichte-Matrix für das gesamte Feld und führt auch die Normalisierung durch. Dabei werden die Gewichte aus dem Online-Othello Programm Cowthello verwendet. Cowthello ist unter der URL <https://www.aurochs.org/games/cowthello/> verfügbar.

In [ ]:
def gen_cowthello_matrix():
    quarter = np.array([
        [100, -25, 25, 10],
        [-25, -50,  1,  1],
        [ 25,   1, 50,  5],
        [ 10,   1,  5,  1]
    ])
    top_half = np.hstack((quarter, np.flip(quarter, axis=1)))
    bottom_half = np.flip(top_half, axis=0)
    raw_matrix = np.vstack((top_half, bottom_half))
    max_possible = np.sum(np.absolute(raw_matrix))
    return np.true_divide(raw_matrix, max_possible)

cowthello_weights = gen_cowthello_matrix()

In [ ]:
def cowthello_heuristic(state):
    return np.sum(np.multiply(state.board, cowthello_weights))

In [ ]:
def safe_in_corner(board, safe, player, rdir, cdir):
    safe_in_row = 9
    rows = range(8) if rdir == 1 else reversed(range(8))
    for row in rows:
        i = np.argmax(board[row,::cdir] != player)
        safe_in_row = min(i, safe_in_row - 1)
        if safe_in_row == 0:
            break
        safe[row,::cdir][:safe_in_row] = True
    safe_in_col = 9
    cols = range(8) if cdir == 1 else reversed(range(8))
    for col in cols:
        i = np.argmax(board[::rdir,col] != player)
        safe_in_col = min(i, safe_in_col - 1)
        if safe_in_col == 0:
            break
        safe[::rdir,col][:safe_in_col] = True

In [ ]:
def safe_pieces(state, player):
    board = state.board
    safe = numpy.zeros((8, 8), dtype=np.bool)
    safe_in_corner(board, safe, player, 1, 1)
    safe_in_corner(board, safe, player, 1,-1)
    safe_in_corner(board, safe, player,-1, 1)
    safe_in_corner(board, safe, player,-1,-1)
    return safe

In [ ]:
def cowthello_safe_heuristic(state):
    black_safe = safe_pieces(state, BLACK)
    white_safe = safe_pieces(state, WHITE)
    weights = numpy.copy(cowthello_weights)
    weights[black_safe] = abs(weights[black_safe])
    weights[white_safe] = abs(weights[white_safe])
    return np.sum(np.multiply(state.board, weights))

Die oben implementierten Heuristiken bewerten jeweils nur ein Merkmal der aktuellen Spielsitation. Durch eine Kombination mehrerer dieser Heuristiken können mehrere Merkmale gleichzeitig betrachtet werden. Die Kombination der Heuristiken kann in Abhängigkeit der aktuellen Spielphase geschehen. Beispielsweise wird zu Ende des Spiels die Disc-Count-Heuristik wichtiger.

In [ ]:
def combined_heuristic(state):
    if state.num_pieces >= 50:
        return disc_count_heuristic(state)
    mobility = combined_mobility_heuristic(state)
    cowthello = cowthello_safe_heuristic(state)
    return 0.5 * mobility + 0.5 * cowthello

## Implementierung der Strategien

### Zufällige KI
Die zufällige KI bewertet den Nutzen aller Züge gleich, gibt also immer den Wert `0` zurück.

In [ ]:
def random_ai(state, depth, heuristic, alpha, beta):
    return 0

### Minimax KI
Diese KI verwendet den Minimax-Algorithmus zur Bestimmung der Nützlichkeit eines Zuges.

In [ ]:
debug_mm_count = 0


def minimax(state, depth, heuristic, alpha, beta):
    global debug_mm_count
    if state.game_over:
        return get_utility(state)
    if depth == 0:
        debug_mm_count += 1
        return heuristic(state)

    if state.turn == WHITE:
        # maximizing
        utility = -math.inf
    else:
        # minimizing
        utility = math.inf

    for move in state.possible_moves:
        tmp_state = make_move(state, move)
        tmp_utility = minimax(tmp_state, depth - 1, heuristic, None, None)
        if state.turn == WHITE:
            # maximizing
            utility = max(utility, tmp_utility)
        else:
            # minimizing
            utility = min(utility, tmp_utility)
    return utility

### Alpha-Beta KI
Diese KI verwended den Minimax Algorithmus mit der Optimierung Alpha-Beta Pruning, um die Nützlichkeit eines Spielzustands zu bestimmen.

Zum Merken vorheriger Ausführungen von wird das Dictionary `transposition_table` verwendet. Dies ist gerade bei der Verwendung von Iterative Deepening für das Move-Ordering vorteilhaft. Der Schlüssel des Dictionaries besteht aus dem Zustand des Spielbretts, dem Spieler, der an der Reihe ist und der verwendeten Heuristik.

In [ ]:
transposition_table = {}

Die Funktion `alphabeta` implementiert den Minimax Algorithmus mit Alpha-Beta-Pruning

debug_ab_count = 0


def alphabeta(state, depth, heuristic, alpha, beta):
    global debug_ab_count
    if state.game_over:
        return get_utility(state)
    if depth == 0:
        debug_ab_count += 1
        return heuristic(state)

    moves = state.possible_moves
    child_states = [make_move(state, move) for move in moves]
    ordered_moves = []
    for child_state in child_states:
        cached = transposition_table.get(
            (child_state.board.tobytes(), child_state.turn, heuristic),
            None
        )
        if cached != None:
            ordered_moves.append((cached, child_state))
        else:
            ordered_moves.append((heuristic(state), child_state))
    ordered_moves.sort(reverse=(state.turn == WHITE))

    if state.turn == WHITE:
        # maximizing
        utility = -math.inf
    else:
        # minimizing
        utility = math.inf

    for (_, tmp_state) in ordered_moves:
        tmp_utility = alphabeta(tmp_state, depth - 1, heuristic, alpha, beta)
        transposition_table[(tmp_state.board.tobytes(),
                             tmp_state.turn, heuristic)] = tmp_utility

        if state.turn == WHITE:
            # maximizing
            utility = max(utility, tmp_utility)
            alpha = max(alpha, utility)
        else:
            # minimizing
            utility = min(utility, tmp_utility)
            beta = min(beta, utility)
        if alpha >= beta:
            break  # alphabeta pruning
    return utility

### ProbCut KI
An dieser Stelle beginnt die Implementierung der Künstlichen Intelligenz mittels des Minimax Algorithmus und ProbCut

In [ ]:
PERCENTILE = 1.5  # 93.3%
PROBCUT_DEEP_DEPTH = 4
PROBCUT_SHALLOW_DEPTH = 2

In [ ]:
debug_pc_count = 0


def probcut(state, depth, heuristic, alpha, beta):
    global debug_pc_count
    if state.game_over:
        return get_utility(state)
    if depth == 0:
        debug_pc_count += 1
        return heuristic(state)

    if depth == PROBCUT_DEEP_DEPTH - 1:
        if state.num_pieces <= 45:
            sigma = state.num_pieces * 0.00160610675723707 + 0.03369559303029926
            # v >= beta with prob. of at least p? yes => cutoff
            bound = PERCENTILE * sigma + beta
            if probcut(state, PROBCUT_SHALLOW_DEPTH,
                       heuristic, bound-1, bound) >= bound:
                return beta

            # v <= alpha with prob. of at least p? yes => cutoff
            bound = -PERCENTILE * sigma + alpha
            if probcut(state, PROBCUT_SHALLOW_DEPTH,
                       heuristic, bound, bound+1) <= bound:
                return alpha

    moves = state.possible_moves
    child_states = [make_move(state, move) for move in moves]
    ordered_moves = []
    for child_state in child_states:
        cached = transposition_table.get(
            (child_state.board.tobytes(), child_state.turn, heuristic), None
        )
        if cached != None:
            ordered_moves.append((cached, child_state))
        else:
            ordered_moves.append((heuristic(state), child_state))
    ordered_moves.sort(reverse=(state.turn == WHITE))

    if state.turn == WHITE:
        # maximizing
        utility = -math.inf
    else:
        # minimizing
        utility = math.inf

    for (_, tmp_state) in ordered_moves:
        tmp_utility = probcut(tmp_state, depth - 1, heuristic, alpha, beta)
        transposition_table[(tmp_state.board.tobytes(),
                             tmp_state.turn, heuristic)] = tmp_utility

        if state.turn == WHITE:
            # maximizing
            utility = max(utility, tmp_utility)
            alpha = max(alpha, utility)
        else:
            # minimizing
            utility = min(utility, tmp_utility)
            beta = min(beta, utility)
        if alpha >= beta:
            break  # alphabeta pruning
    return utility

### Durchführen der Züge
Die folgenden Funktionen berechnen mit einer angegebenen KI den nächsten Zug und wenden diesen auf den Zustand an. Der Wert `SELECTION_TOLERANCE` gibt an, wie viel der Nutzen eines Zuges vom besten Nutzen abweichen darf, um dennoch ausgewählt werden zu können.

In [ ]:
SELECTION_TOLERANCE = 0.0025

Die Funktion `ai_make_move` führt auf dem Zustand `state` den besten, durch den Algorithmus `ai` bestimmten, Zug aus.
Für alle möglichen Züge wird die Nützlichkeit des resultierenden Zustands mithilfe des Algorithmus `ai` bestimmt. Aus allen Zügen wird einer der Züge ausgewählt, der für den Spieler den optimaleln Nutzen hat.

In [ ]:
def ai_make_move(ai, state, depth, heuristic):
    global utilities
    if state.game_over:
        return
    scored_moves = []
    if state.turn == WHITE:
        # maximizing
        alpha = -math.inf
        for move in state.possible_moves:
            new_state = make_move(state, move)
            utility = ai(new_state, depth-1, heuristic, alpha, math.inf)
            scored_moves.append((utility, move))
            alpha = max(alpha, utility)
        best_score, _ = max(scored_moves)
    else:
        # minimizing
        beta = math.inf
        for move in state.possible_moves:
            new_state = make_move(state, move)
            utility = ai(new_state, depth-1, heuristic, -math.inf, beta)
            scored_moves.append((utility, move))
            beta = min(beta, utility)
        best_score, _ = min(scored_moves)
    utilities[state.turn] = best_score
    top_moves = [move for move in scored_moves if abs(move[0] - best_score) <= SELECTION_TOLERANCE]
    best_move = random.choice(top_moves)[1]
    return make_move(state, best_move)

Beim Iterative Deepening wird der Alphabeta-Algorithmus nacheinander mit steigender Tiefe ausgeführt. Die Werte aus der vorherigen Ausführung werden dann für das Move-Ordering verwendet. Dadurch das bessere Move-Ordering können beim Alpha-Beta-Pruning mehr Zweige abgeschnitten werden. Dadurch kann trotz der mehrfachen Ausführung eine höhere Performanz bei gleichem Ergebnis erreicht werden. Ein weiterer Vorteil ist, dass das Iterative Deepening jederzeit abgebrochen werden kann. Dadurch kann in einer vorgegebenen Zeit, die in dieser Zeit maximal mögliche Suchtiefe erreicht werden.

In [ ]:
def ai_make_move_id(ai, state, depth, heuristic):
    global utilities
    if state.game_over:
        return
    best_move = None
    cur_depth = 1
    while cur_depth <= depth:
        scored_moves = []
        if state.turn == WHITE:
            # maximizing
            alpha = -math.inf
            for move in state.possible_moves:
                new_state = make_move(state, move)
                utility = ai(new_state, cur_depth-1, heuristic, alpha, math.inf)
                scored_moves.append((utility, move))
                alpha = max(alpha, utility)
            best_score, _ = max(scored_moves)
        else:
            # minimizing
            beta = math.inf
            for move in state.possible_moves:
                new_state = make_move(state, move)
                utility = ai(new_state, cur_depth-1, heuristic, -math.inf, beta)
                scored_moves.append((utility, move))
                beta = min(beta, utility)
            best_score, _ = min(scored_moves)
        utilities[state.turn] = best_score
        top_moves = [move for move in scored_moves
                     if abs(move[0] - best_score) <= SELECTION_TOLERANCE]
        best_move = random.choice(top_moves)[1]
        cur_depth += 1
    return make_move(state, best_move)

Beim Iterative Deepening ist jederzeit der beste Zug der letzten Suchtiefe bekannt. Daher kann der Algorithmus verwendet werden, um in einer vorgegebenen Zeit mit größtmöglicher Suchtiefe zu suchen. Dazu wird die Dauer des nächsten Zugs anhand der Dauer des letzten Zugs geschätzt. Somit kann es vorkommen, dass die Berechnung etwas länger oder kürzer als die gegebene Zeit dauert. Dafür wird das Ergebnis aller Berechnungen verwendet und die letzte Suche muss nicht abgebrochen werden.

In [ ]:
SECS_PER_MOVE = 30


def ai_make_move_id_timelimited(ai, state, depth, heuristic):
    global utilities
    if state.game_over:
        return
    best_move = None
    depth = 1
    last_time = 1
    second_last_time = 1
    factor = 0
    start = time.time()
    while (
        depth <= 64 - state.num_pieces and
        SECS_PER_MOVE - (time.time() - start) >= factor * last_time
    ):
        last_time_start = time.time()
        scored_moves = []
        if state.turn == WHITE:
            # maximizing
            alpha = -math.inf
            for move in state.possible_moves:
                new_state = make_move(state, move)
                utility = ai(new_state, depth-1, heuristic, alpha, math.inf)
                scored_moves.append((utility, move))
                alpha = max(alpha, utility)
            best_score, _ = max(scored_moves)
        else:
            # minimizing
            beta = math.inf
            for move in state.possible_moves:
                new_state = make_move(state, move)
                utility = ai(new_state, depth-1, heuristic, -math.inf, beta)
                scored_moves.append((utility, move))
                beta = min(beta, utility)
            best_score, _ = min(scored_moves)
        utilities[state.turn] = best_score
        top_moves = [move for move in scored_moves
                     if abs(move[0] - best_score) <= SELECTION_TOLERANCE]
        best_move = random.choice(top_moves)[1]
        second_last_time = last_time
        last_time = time.time() - last_time_start
        factor = min(last_time / second_last_time, 3)
        depth += 1
    print("Reached depth", depth-1, "in", time.time() - start, "seconds")
    return make_move(state, best_move)